# Estimation of paramters in the probit model with an 'exact' Jeffreys

This script allows the conduction of *a posteriori* sampling of the paramter in the probit statistical model with a finely approximated Jeffreys prior.


The derivation of the Jeffreys prior for the probit model is proposed in the repository [bayes_frag](https://github.com/vbkantoine/bayes_frag). That external code also allows to directly generate samples from the posterior yielded by that approximated Jeffreys prior.


### 1. Clone the bayes_frag github

In [1]:
# 1. clone github bayes_frag

import os
import sys

if not os.path.exists('bayes_frag') :
    !git clone "https://github.com/vbkantoine/bayes_frag.git"

sys.path.append(os.path.join(os.getcwd(),'bayes_frag'))

Cloning into 'bayes_frag'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 74 (delta 32), reused 64 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 37.03 KiB | 7.41 MiB/s, done.
Resolving deltas: 100% (32/32), done.


### 2. Compute the Jeffreys prior for this model

We recall that the probit model is defined as a parametrized model where $\theta=(\theta_1,\theta_2)\in(0,\infty)^2$ is the parameter and the observed variable is $(Z,a)$ where 
$$\left\lbrace\begin{array}{l}a\sim\mathrm{Log-}\mathcal{N}(\mu_a,\sigma_a^2)\\
Z\sim\mathrm{Bernoulli}(P_f(a))\end{array}\right.,$$
with $P_f(a)=\Phi\left(\frac{\log a-\log\theta_1}{\theta_2}\right)$, and $\Phi$ denoting the c.d.f. of a standard Gaussian.

The Jeffreys prior of this model depends on the distribution of $a$, i.e. it depends on $\mu_a$ and $\sigma_a$.

That external repository proposes a code to compute the Jeffreys prior given a distribution of $a$. Actually, it derives a fine numerical approximation of the Fisher information matrix that is stored in a file called `fisher`.

The exection of that code is generally very long given the complex expression of the Fisher information matrix.
For this reason, we suggest to download the one that we have already computed and that we provide online on [OSF](https://osf.io/gvqw4/files/osfstorage/678a826e9b2975f377dd6f3f). The dowloaded file `fisher` can be placed at the root of the current directory.

We also have provided a lighter approximation of the Fisher information matrix based on a less thin derivation. It is stored in the file called `fisher_light`, that can be renamed by `fisher` to be used. 

If one wants to do the computations instead of downloading the `fisher` file or renaming the `fisher_light`file, the last line of the following cell must be uncommented. 


In [2]:
# 2. compute and save a fine mesh of Jeffreys prior

import util.create_fisher_artificial as cfa

dat = cfa.Data_simplified_big()

def save_fisher_computation():
    save_fisher_arr = cfa.dict_save_fisher['save_fisher_arr'] 
    # selecting the dictionnary cfa.dict_save_fisher2 above lead to a lighter appproximation of Fisher 
    # (the one that we have provided and called `fisher_light`)

    alpha_min = save_fisher_arr.alpha_min
    alpha_max = save_fisher_arr.alpha_max
    beta_min = save_fisher_arr.beta_min
    beta_max = save_fisher_arr.beta_max
    num = save_fisher_arr.num_alpha

    theta_tab1 = save_fisher_arr.alpha_tab
    theta_tab2 = save_fisher_arr.beta_tab

    function = cfa.fisher.fisher_function("simpson", dat)
    I = cfa.save_fisher(cfa.save_path, function, theta_tab1, theta_tab2)
    
# # save_fisher_computation()


### 3. Conduction of *a posteriori* sampling

In the following, we import data from the file `../tirages_data` to derive a posterior that is used to generate samples of the paramter $\theta$.
Then, the results are saved on different files according to the prior: Jeffreys or the constrained Jeffreys.



In [3]:
import os
import numpy as np
import pickle
from numba import jit

from bayes_frag import stat_functions as stat_f
from bayes_frag.data import Data
from bayes_frag.model import Model
from bayes_frag import config
from bayes_frag.reference_curves import Reference_known_MLE
from bayes_frag.extract_saved_fisher import Approx_fisher

from util.create_fisher_artificial import dict_save_fisher

In [4]:
class Data_simplified(Data) :
    """
       This class serves the import of data contained in an external file 
    """
    def __init__(self, i, pickle_path):
        """
        Args:
            i (int): in the generated array, id to take into account in this run
        """
        file = open(pickle_path, mode='rb')
        array_A_Z = pickle.load(file)
        self.A = np.array(array_A_Z[0][:,i,np.newaxis], dtype=np.float, order='C')
        self.Z = np.array(array_A_Z[1][:,i,np.newaxis], dtype=np.float, order="C")
        self.Y = np.ones_like(self.A)
        self.a_tab = None
        self.h_a = None
        self._set_a_tab()
        self.f_A = None
        self.f_A_tab = None
        self._compute_f_A()
        self.increasing_mode = True

In [5]:

kappa = 1/8
alpha = 1/2
gamma = kappa/alpha

save_folder = './'
path_tirages_data = '../tirages_probit/'

theta_vrai = np.array([3.37610525, 0.43304097])


def experiment(N) :
    init_seed = 0
    SEED = init_seed + int(N)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    assert not os.path.exists(os.path.join(save_folder, 'model_J_constraint_{}'.format(N-1))), 'existing run no {}'.format(N)

    i_data = N-1

    data = Data_simplified(i_data, os.path.join(path_tirages_data, 'tirages_data'))

    # print(data.A.shape)
    # print(data.Z)
    error_draw = np.arange(1,41)*5
    n_est = 5000

    ##
    # parameters models
    linear = False
    approx_fisher_class = Approx_fisher(dict_save_fisher['save_fisher_path'], dict_save_fisher['save_fisher_arr'], fisher_file_path_is_personalized=True)
    fisher =  approx_fisher_class.fisher_approx
    likelihood = stat_f.log_vrais

    iter_HM = 15000
    sigma_prop = np.array([[0.1,0],[0,0.095]])
    t0 = np.array([3,0.3])
    HM_fun = stat_f.adaptative_HM


    @jit
    def prior(th) :
        I = fisher(th)
        return 1/2 * np.log(np.nan_to_num(np.abs(I[:,0,0]*I[:,1,1]-I[:,0,1]**2)))

    # log_post for this case :
    @jit(nopython=True)
    def log_post(z,a, theta) :
        return stat_f.log_post_jeff_adapt(theta,z,a, Fisher=fisher)

    @jit
    def prior_adapted(th) :
        return prior(th) + gamma * np.log(th[:,1]*th[:,1])

    @jit(nopython=True)
    def log_post_adapted(z,a,theta) :
        return log_post(z,a,theta) + gamma*np.log(theta[:,1])

    ref = Reference_known_MLE(data, theta_vrai)
    
    HM_post_simul_constraint = stat_f.Post_HM_Estimator(HM_fun, t0, log_post_adapted, pi_log=True, max_iter=iter_HM, sigma0=sigma_prop)
    model_J_constraint = Model(prior_adapted, likelihood, data, HM_post_simul_constraint, linear=linear, ref=ref)

    HM_post_simul = stat_f.Post_HM_Estimator(HM_fun, t0, log_post, pi_log=True, max_iter=iter_HM, sigma0=sigma_prop)
    model_J = Model(prior, likelihood, data, HM_post_simul, linear=linear, ref=ref)

    
    ## run everything

    model_J_constraint.run_simulations(error_draw, n_est, sim=['post'], print_fo=True)
    model_J.run_simulations(error_draw, n_est, sim=['post'], print_fo=True)

    pickle.dump({'logs': model_J_constraint.logs, 'A':model_J_constraint.A, 'S':model_J_constraint.S, 'seed':SEED}, open(os.path.join(save_folder, "model_J_constraint_{}".format(i_data)), "wb"))
    pickle.dump({'logs': model_J.logs, 'A':model_J.A, 'S':model_J.S, 'seed':SEED}, open(os.path.join(save_folder, "model_J_{}".format(i_data)), "wb"))




The above function can be ran several time to get samples given different samples of the data.
The following permits to conduct $10$ oh these experiments. 
It re-generates the files `model_J_*` that are provided in the directory.
We warn the user that this execution can take a long time.

In [ ]:
# 4. (long code) all the following run section 3. for 10 different seeds

execute_loop = False # True to execute the following

if execute_loop :
    for N in range(1,11) :
        print('****** Experiment no {}/10 ******'.format(N))
        experiment(N)

****** Experiment no 1/10 ******
Simulation 1/40, dataset_size=5, simulating ['post']


/var/folders/q2/pg0k5xqs0r9brvk6cxh1hb_m0000gn/T/ipykernel_16765/3424857049.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.A = np.array(array_A_Z[0][:,i,np.newaxis], dtype=np.float, order='C')
/var/folders/q2/pg0k5xqs0r9brvk6cxh1hb_m0000gn/T/ipykernel_16765/3424857049.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.Z 

Simulation 2/40, dataset_size=10, simulating ['post']
Simulation 3/40, dataset_size=15, simulating ['post']
Simulation 4/40, dataset_size=20, simulating ['post']
Simulation 5/40, dataset_size=25, simulating ['post']
Simulation 6/40, dataset_size=30, simulating ['post']
Simulation 7/40, dataset_size=35, simulating ['post']
Simulation 8/40, dataset_size=40, simulating ['post']
Simulation 9/40, dataset_size=45, simulating ['post']
Simulation 10/40, dataset_size=50, simulating ['post']
Simulation 11/40, dataset_size=55, simulating ['post']
Simulation 12/40, dataset_size=60, simulating ['post']
Simulation 13/40, dataset_size=65, simulating ['post']
Simulation 14/40, dataset_size=70, simulating ['post']
Simulation 15/40, dataset_size=75, simulating ['post']
Simulation 16/40, dataset_size=80, simulating ['post']
Simulation 17/40, dataset_size=85, simulating ['post']
Simulation 18/40, dataset_size=90, simulating ['post']
Simulation 19/40, dataset_size=95, simulating ['post']
Simulation 20/40, 

/var/folders/q2/pg0k5xqs0r9brvk6cxh1hb_m0000gn/T/ipykernel_16765/3424857049.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.A = np.array(array_A_Z[0][:,i,np.newaxis], dtype=np.float, order='C')
/var/folders/q2/pg0k5xqs0r9brvk6cxh1hb_m0000gn/T/ipykernel_16765/3424857049.py:13: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.Z 

Simulation 2/40, dataset_size=10, simulating ['post']
Simulation 3/40, dataset_size=15, simulating ['post']
Simulation 4/40, dataset_size=20, simulating ['post']
Simulation 5/40, dataset_size=25, simulating ['post']
Simulation 6/40, dataset_size=30, simulating ['post']
Simulation 7/40, dataset_size=35, simulating ['post']
Simulation 8/40, dataset_size=40, simulating ['post']
Simulation 9/40, dataset_size=45, simulating ['post']
Simulation 10/40, dataset_size=50, simulating ['post']
